## Import Packages & Function Definitions

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextGenerationPipeline
import torch
import accelerate

def get_pipeline(path:str, tokenizer:AutoTokenizer, accelerator:accelerate.Accelerator) -> TextGenerationPipeline:
    model = AutoModelForCausalLM.from_pretrained(
        path, torch_dtype=torch.float16, device_map='auto', trust_remote_code=True)
    
    terminators = [tokenizer.eos_token_id, tokenizer.pad_token_id]

    pipeline = TextGenerationPipeline(model = model, tokenizer = tokenizer, num_workers=accelerator.state.num_processes*4, pad_token_id=tokenizer.pad_token_id, eos_token_id=terminators)

    return pipeline

## Taigi-Llama-2-series: Causal Language Modeling for Taigi

In [2]:
model_dir = "Bohanlu/Taigi-Llama-2-7B" # or Bohanlu/Taigi-Llama-2-13B for the 13B model
tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=False)

accelerator = accelerate.Accelerator()
pipe = get_pipeline(model_dir, tokenizer, accelerator)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
# CausalLM示例：台語文本生成
clm_prompt = "我真愛食台灣的"

# Few-shot示例：情感分析
sentiment_prompt = """Example 1:
輸入：這齣電影真正是蓋讚啦！
情感：正面

Example 2:
輸入：今仔日的天氣真䆀。
情感：負面

Example 3:
輸入：這間餐廳的服務一般般仔爾爾。
情感：中性

Example 4:
輸入：我拄買彼支手機仔的螢幕誠大塊。
情感："""

# Few-shot示例：問答
qa_prompt = """Example 1:
問題：台北101有偌懸？
答案：台北101的高度是五百空八公尺。

Example 2:
問題：台灣上長的溪仔是佗一條？
答案：台灣上長的溪仔是濁水溪，規个長度有百八公里遐爾長。

Example 3:
問題：臺灣上懸的山是啥物？
答案："""

# Few-shot示例：台語翻譯
translation_prompt = """Example 1:
中文：你好嗎？
台語：你好無？

Example 2:
中文：我很喜歡吃水果。
台語：我真愛食水果。

Example 3:
中文：是否有人會講台語？
台語：敢有人會曉講臺語？

Example 4:
中文：請問這裡怎麼走到火車站？
台語："""

In [4]:
pipe([clm_prompt, sentiment_prompt, qa_prompt, translation_prompt], return_full_text=False)

[[{'generated_text': '水果，'}],
 [{'generated_text': '正面'}],
 [{'generated_text': '臺灣上懸的山是玉山，伊的懸度是三千九百五十二公尺。'}],
 [{'generated_text': '請問這欲按怎去到火車站？'}]]

## Taigi-Llama-2-Translator-series: A Comprehensive Translator for Traditional Chinese, English, and Taigi (POJ, Hanzi and Hanlo)

In [2]:
model_dir = "Bohanlu/Taigi-Llama-2-Translator-7B" # or Bohanlu/Taigi-Llama-2-Translator-13B for the 13B model
tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=False)

accelerator = accelerate.Accelerator()
pipe = get_pipeline(model_dir, tokenizer, accelerator)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
PROMPT_TEMPLATE = "[TRANS]\n{source_sentence}\n[/TRANS]\n[{target_language}]\n"
def translate(source_sentence:str, target_language:str) -> str:
    prompt = PROMPT_TEMPLATE.format(source_sentence=source_sentence, target_language=target_language)
    out = pipe(prompt, return_full_text=False, repetition_penalty=1.1, do_sample=False)[0]['generated_text']
    return out[:out.find("[/")].strip()

source_sentence = "How are you today？"

print(f"{source_sentence=}\n")
print("To Hanzi: " + translate(source_sentence, "HAN"))
print("To POJ: " + translate(source_sentence, "POJ"))
print("To Traditional Chinese: " + translate(source_sentence, "ZH"))
print("To Hanlo: " + translate(source_sentence, "HL"))

source_sentence='How are you today？'



/home/bohan1234/anaconda3/envs/twllm/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/bohan1234/anaconda3/envs/twllm/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


To Hanzi: 你今仔日好無？
To POJ: Lí kin-á-ji̍t án-chóaⁿ?
To Traditional Chinese: 你今天好嗎？
To Hanlo: 你今仔日好無？


In [7]:
source_sentence = "Thài-khong pêng-iú, lín hó! Lín chia̍h-pá--bē?"

print(f"{source_sentence=}\n")
print("To Hanzi: " + translate(source_sentence, "HAN"))
print("To English: " + translate(source_sentence, "EN"))
print("To Traditional Chinese: " + translate(source_sentence, "ZH"))
print("To Hanlo: " + translate(source_sentence, "HL"))

source_sentence='Thài-khong pêng-iú, lín hó! Lín chia̍h-pá--bē?'

To Hanzi: 太空朋友，恁好！恁食飽未？
To English: Space friends, you guys are great! Have you eaten yet?
To Traditional Chinese: 太空朋友，你們好！你們吃飽了嗎？
To Hanlo: 太空朋友，lín好！Lín食飽--未？


## Taigi-Llama-2-Chat Series: A Comprehensive Chat Model for Taigi

In [3]:
model_dir = "Bohanlu/Taigi-Llama-2-Chat-7B" # or Bohanlu/Taigi-Llama-2-Chat-13B for the 13B model
tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=False)

accelerator = accelerate.Accelerator()
pipe = get_pipeline(model_dir, tokenizer, accelerator)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
# Single round
messages = [
    {"role": "user", "content": "你做一个百歲老樹，佇青翠的森林內底，安養天年，雄雄一陣剉樹仔的人衝過來欲共你剉掉，按呢你會有啥物款的心情？"},
]

pipe(messages, return_full_text=False, repetition_penalty=1.05, do_sample=True)[0]['generated_text']

' 做一欉千年老樹，佇青翠的森林中安養天年，雄雄一陣剉樹仔的人衝過來欲共我剉掉，這陣我心肝頭會真毋甘、真袂慣勢。\n\n頭先，我感覺真無歡喜佮袂慣勢。我佇遮徛幾若百年矣，早就成做森林的一部份，予人當做是寶物咧保護。今仔日煞有人來破壞我的安寧，予我感覺足無公平閣艱苦。\n\n閣來，我煩惱家己的性命會結束。做一欉大樹，我的性命佮森林是相黏牢的。當我予人剉掉的時陣，嘛代表規个森林生態系統的破壞。我驚惶森林的生物會失去我的庇護，面對生存危機。\n\n毋過，我嘛知影這个世界一直攏咧改變，人類嘛一直咧發展。佇面對改變的時陣，咱愛保持彈性佮適應力。我希望人會當尊重大自然的法則，保護生態系統，予地球會當永續發展。\n\n總講一句，我希望人會當理解樹仔對生態系統的重要性，嘛愛尊重大自然的平衡。我雖然是樹仔，嘛是生態系統的一份子，我希望所有的生物攏會當和平共存，做伙維護這个媠閣可愛的地球。 '

In [8]:
# Multiple rounds
messages = [
    {"role": "user", "content": "你做一个百歲老樹，佇青翠的森林內底，安養天年，雄雄一陣剉樹仔的人衝過來欲共你剉掉，按呢你會有啥物款的心情？"},
    {"role": "assistant", "content": "做一欉千年老樹，佇青翠的森林中安養天年，雄雄一陣剉樹仔的人衝過來欲共我剉掉，這陣我心肝頭會真毋甘、真袂慣勢。\n\n頭先，我感覺真無歡喜佮袂慣勢。我佇遮徛幾若百年矣，早就成做森林的一部份，予人當做是寶物咧保護。今仔日煞有人來破壞我的安寧，予我感覺足無公平閣艱苦。\n\n閣來，我煩惱家己的性命會結束。做一欉大樹，我的性命佮森林是相黏牢的。當我予人剉掉的時陣，嘛代表規个森林生態系統的破壞。我驚惶森林的生物會失去我的庇護，面對生存危機。\n\n毋過，我嘛知影這个世界一直攏咧改變，人類嘛一直咧發展。佇面對改變的時陣，咱愛保持彈性佮適應力。我希望人會當尊重大自然的法則，保護生態系統，予地球會當永續發展。\n\n總講一句，我希望人會當理解樹仔對生態系統的重要性，嘛愛尊重大自然的平衡。我雖然是樹仔，嘛是生態系統的一份子，我希望所有的生物攏會當和平共存，做伙維護這个媠閣可愛的地球。"},
    {"role": "user", "content": "提出挽救森林的方案，來說服剉樹仔的人停手莫閣剉矣。"},
]

pipe(messages, return_full_text=False, repetition_penalty=1.05, do_sample=True)[0]['generated_text']

' 做一欉千年老樹，我深知森林對生態系的重要性，嘛了解人類佮大自然之間的互相影響。為著拯救森林，我提出下列幾點建議，向望會當說服剉樹仔的人停手莫閣剉矣：\n\n1. 永續林業：推廣永續林業的理念，透過合理的砍伐佮種樹計畫，維持森林的生態平衡。予森林佇會當繼續發展的前提之下，提供資源予人類使用。\n2. 生態觀光：開發森林生態觀光，予遊客體驗森林的美麗，同時也增加當地居民的收入。按呢會使減少剉樹仔的壓力，同時也會當保護森林的環境。\n3. 種樹造林：鼓勵人佇都市抑是廢墟的土地頂懸種樹仔，來恢復森林的面積。毋但會當美化環境，閣會當改善空氣的品質，調節氣候。\n4. 加強森林保護：建立森林保護區，限制人入去森林採集資源，禁止違法剉樹仔。政府佮民間愛共同拍拚，加強對森林的保護。\n5. 教育佮宣導：透過教育佮宣導，提高民眾對森林生態的重要性認捌。予閣較濟人了解森林對人類的重要性，從而支持保護森林的行動。\n6. 科技創新：利用科技創新，研發較環保的木材加工技術，降低對森林資源的損耗。同時，嘛會使開發再生能源，親像太陽能、風力發電等等，來代替對森林資源的依賴。\n\n透過遮的建議，我希望會當喚起人的意識，予𪜶知影保護森林的重要。我相信，只要人會當理解大自然的平衡佮永續發展的重要性，就一定會停止剉樹仔的行為，做伙來保護這个美麗的地球。 '